In [19]:
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from typing import List, Tuple, Optional, Any



GRID_ROWS: int = 50
GRID_COLUMNS: int = 50

NUM_PARTICLES_PERCENTAGE: float = 0.05 

SIMULATION_STEPS: int = 100

ANIMATION_INTERVAL_MS: int = 300  

BOUNDARY_TYPE: str = 'wall'  


ANIMATION_FILENAME: str = "fhp_simulation.gif" 

# ANIMATION_FILENAME: str = "fhp_simulation_v3_circular_particles_red.mp4"

# Set a seed for reproducible simulations. Set to None for random behavior.
RANDOM_SEED: Optional[int] = 42


class Node:
    
    """Represents a single cell in the lattice."""
    
    def __init__(self, is_wall: bool = False):
        self.ni_s: List[int] = [0] * 6
        self.is_wall: bool = is_wall
        self.occupied: int = 1 if is_wall else 0

    def __repr__(self) -> str:
        return f"Node(wall={self.is_wall}, ni_s={self.ni_s}, occupied={self.occupied})"


def initialize_lattice(rows: int, cols: int, boundary_type: str) -> np.ndarray:
    
    """Initializes the simulation lattice with Node objects."""
    lattice = np.array([[Node(is_wall=False) for _ in range(cols)] for _ in range(rows)], dtype=object)
    
    if boundary_type == 'wall':
        for r_idx in range(rows):
            for c_idx in range(cols):
                if r_idx == 0 or r_idx == rows - 1 or c_idx == 0 or c_idx == cols - 1:
                    lattice[r_idx, c_idx] = Node(is_wall=True)
    return lattice

# --- Neighbor Finding ---
def get_hexagonal_neighbors(coords: Tuple[int, int], rows: int, cols: int) -> List[Tuple[int, int]]:
    """Gets the 6 neighbors of a cell in a hexagonal layout on a square grid."""
    r, c = coords
    return [
        (r, (c + 1) % cols),
        ((r - 1 + rows) % rows, (c + 1) % cols),
        ((r - 1 + rows) % rows, (c - 1 + cols) % cols),
        (r, (c - 1 + cols) % cols),
        ((r + 1) % rows, (c - 1 + cols) % cols),
        ((r + 1) % rows, (c + 1) % cols)
    ]


# --- Particle Initialization ---
def fill_lattice_randomly(
    lattice: np.ndarray, 
    num_particles_percentage: float, 
    rows: int, 
    cols: int, 
    boundary_type: str
    ):
    """
    Fills the lattice randomly with particles across the entire grid.
    Particles occupy unique (cell_row, cell_col, direction_index) slots.
    The number of particles is determined by num_particles_percentage of available slots.
    """
    possible_slots: List[Tuple[int, int, int]] = []
    
    for r_idx in range(rows):
        for c_idx in range(cols):
            if boundary_type == 'wall' and lattice[r_idx, c_idx].is_wall:
                continue
            for direction_idx in range(6):
                possible_slots.append((r_idx, c_idx, direction_idx))
    
    if not possible_slots:
        print("Warning: No available slots to place particles.")
        return

    num_particles_to_place = int(num_particles_percentage * len(possible_slots))
    if num_particles_to_place == 0 and num_particles_percentage > 0 and possible_slots:
        num_particles_to_place = 1
        
    actual_num_particles = min(num_particles_to_place, len(possible_slots))

    if actual_num_particles > 0:
        chosen_slots = random.sample(possible_slots, actual_num_particles)
        for r_idx, c_idx, direction_idx in chosen_slots:
            lattice[r_idx, c_idx].ni_s[direction_idx] = 1
            lattice[r_idx, c_idx].occupied = 1
    
    print(f"Initialized {actual_num_particles} particles across the grid (out of {len(possible_slots)} available slots, {num_particles_percentage*100:.2f}% target density).")


# --- FHP-II Collision Rules ---
def _d_predicate(ni_s: List[int], k: int) -> int:
    
    if ni_s[k % 6] == 1 and \
       ni_s[(k + 3) % 6] == 1 and \
       ni_s[(k + 1) % 6] == 0 and \
       ni_s[(k + 2) % 6] == 0 and \
       ni_s[(k + 4) % 6] == 0 and \
       ni_s[(k + 5) % 6] == 0:
        return 1
    return 0



def _t_predicate(ni_s: List[int], k: int) -> int:
    if ni_s[k % 6] == 1 and \
       ni_s[(k + 2) % 6] == 1 and \
       ni_s[(k + 4) % 6] == 1 and \
       ni_s[(k + 1) % 6] == 0 and \
       ni_s[(k + 3) % 6] == 0 and \
       ni_s[(k + 5) % 6] == 0:
        return 1
    return 0


def calculate_collision_factor_omega(ni_s: List[int], i: int) -> int:
    q = random.choice([0, 1])
    loss_from_d_collision_on_i_axis = _d_predicate(ni_s, i)
    gain_from_d_collision_on_i_minus_1_axis = q * _d_predicate(ni_s, (i - 1 + 6) % 6)
    gain_from_d_collision_on_i_plus_1_axis = (1 - q) * _d_predicate(ni_s, (i + 1 + 6) % 6)
    loss_from_t_collision_involving_i = _t_predicate(ni_s, i)
    gain_from_t_collision_involving_i_plus_3 = _t_predicate(ni_s, (i + 3 + 6) % 6)
    omega = (
        -loss_from_d_collision_on_i_axis +
        gain_from_d_collision_on_i_minus_1_axis +
        gain_from_d_collision_on_i_plus_1_axis -
        loss_from_t_collision_involving_i +
        gain_from_t_collision_involving_i_plus_3
    )
    return int(omega)




# --- Simulation Step ---
def simulation_step(current_lattice: np.ndarray, rows: int, cols: int, boundary_type: str) -> np.ndarray:
    """Performs one step of the FHP simulation (collision and propagation)."""
    next_lattice = initialize_lattice(rows, cols, boundary_type)
    for r_idx in range(rows):
        for c_idx in range(cols):
            current_node = current_lattice[r_idx, c_idx]
            if boundary_type == 'wall' and current_node.is_wall:
                continue
            current_ni_s = current_node.ni_s
            ni_s_post_collision = [0] * 6
            for k_direction in range(6):
                omega_k = calculate_collision_factor_omega(current_ni_s, k_direction)
                particles_in_k_after_collision = current_ni_s[k_direction] + omega_k
                if particles_in_k_after_collision < 0: particles_in_k_after_collision = 0
                if particles_in_k_after_collision > 1: particles_in_k_after_collision = 1
                ni_s_post_collision[k_direction] = particles_in_k_after_collision
            
            neighbor_coords_list = get_hexagonal_neighbors((r_idx, c_idx), rows, cols)
            for k_direction in range(6):
                if ni_s_post_collision[k_direction] == 1:
                    dest_r, dest_c = neighbor_coords_list[k_direction]
                    if boundary_type == 'wall' and next_lattice[dest_r, dest_c].is_wall:
                        next_lattice[r_idx, c_idx].ni_s[(k_direction + 3) % 6] = 1
                    else:
                        next_lattice[dest_r, dest_c].ni_s[k_direction] = 1

    for r_idx in range(rows):
        for c_idx in range(cols):
            node_to_update = next_lattice[r_idx, c_idx]
            if not node_to_update.is_wall:
                if any(particle_present == 1 for particle_present in node_to_update.ni_s):
                    node_to_update.occupied = 1
                else:
                    node_to_update.occupied = 0
    return next_lattice

# --- Visualization and Animation ---
def get_particle_coordinates(lattice: np.ndarray, rows: int, cols: int) -> Tuple[List[int], List[int]]:
    """Extracts x (column) and y (row) coordinates of occupied cells (particles)."""
    x_coords: List[int] = []
    y_coords: List[int] = []
    for r_idx in range(rows):
        for c_idx in range(cols):
            if lattice[r_idx, c_idx].occupied == 1 and not lattice[r_idx, c_idx].is_wall:
                x_coords.append(c_idx) # x is column
                y_coords.append(r_idx) # y is row
    return x_coords, y_coords

def display_lattice_state(lattice: np.ndarray, rows: int, cols: int, step_num: int = -1):
    """Displays a single state of the lattice using Matplotlib scatter plot for circular particles."""
    x_coords, y_coords = get_particle_coordinates(lattice, rows, cols)
    
    plt.figure(figsize=(7, 6))
    ax = plt.gca() # Get current axes
    
    # Plot walls if any (simple imshow for walls)
    wall_grid = np.zeros((rows, cols), dtype=np.int8)
    for r_idx in range(rows):
        for c_idx in range(cols):
            if lattice[r_idx, c_idx].is_wall:
                 wall_grid[r_idx,c_idx] = 1
    # Use a light gray for walls, imshow renders from top-left
    ax.imshow(wall_grid, cmap='Greys', vmin=0, vmax=5, alpha=0.3, origin='upper', extent=[-0.5, cols-0.5, rows-0.5, -0.5])


    # Scatter plot for particles
    # Adjust marker size 's' as needed. It's in points^2.
    # A reasonable size depends on the grid density.
    marker_area = (0.8 * min(700/cols, 600/rows))**2 # Heuristic for marker size
    ax.scatter(x_coords, y_coords, s=marker_area, c='red', marker='o')
    
    ax.set_xlim(-0.5, cols - 0.5)
    # Invert y-axis to match imshow's typical origin (0,0 at top-left)
    ax.set_ylim(rows - 0.5, -0.5) 
    ax.set_aspect('equal', adjustable='box') # Ensure circles are circular

    title = "FHP Lattice State (Circular Particles)"
    if step_num >= 0:
        title += f" - Step {step_num}"
    ax.set_title(title)
    ax.set_xlabel("Column Index")
    ax.set_ylabel("Row Index")
    plt.tight_layout()
    plt.show()

def build_and_save_animation(
    lattice_states: List[np.ndarray], # List of full lattice states (Node objects)
    rows: int,
    cols: int,
    filename: str = "fhp_simulation.gif", 
    interval_ms: int = 150,
    particle_color: str = 'red'
    ):
    """Builds and saves an animation from a list of lattice states using scatter plot."""
    if not lattice_states:
        print("Error: No states provided for animation.")
        return

    fig, ax = plt.subplots(figsize=(7, 6))
    
    # Initial empty scatter plot for particles
    # Marker size calculation (heuristic)
    marker_area = (0.8 * min(700/cols, 600/rows))**2 
    scatter_plot = ax.scatter([], [], s=marker_area, c=particle_color, marker='o')

    # Plot walls once as a background (if boundary_type is 'wall')
    # This assumes walls don't change during simulation.
    wall_grid = np.zeros((rows, cols), dtype=np.int8)
    # Check the first state for walls, assuming they are static.
    # This is a simplification; if walls could change, this would need to be in update_frame.
    if BOUNDARY_TYPE == 'wall': # Access global BOUNDARY_TYPE
        for r_idx in range(rows):
            for c_idx in range(cols):
                if lattice_states[0][r_idx, c_idx].is_wall: # Check the first state
                    wall_grid[r_idx,c_idx] = 1
    # imshow renders from top-left (0,0) by default unless origin is changed
    ax.imshow(wall_grid, cmap='Greys', vmin=0, vmax=5, alpha=0.3, origin='upper', extent=[-0.5, cols-0.5, rows-0.5, -0.5])


    ax.set_xlim(-0.5, cols - 0.5)
    ax.set_ylim(rows - 0.5, -0.5) # Inverted y-axis
    ax.set_aspect('equal', adjustable='box')
    ax.set_title(f"FHP Simulation - Step 0")
    ax.set_xlabel("Column Index")
    ax.set_ylabel("Row Index")
    plt.tight_layout()

    def update_frame(frame_num: int):
        current_lattice = lattice_states[frame_num]
        x_coords, y_coords = get_particle_coordinates(current_lattice, rows, cols)
        
        # Update scatter plot data
        # np.c_ stacks 1D arrays as columns into a 2D array
        if x_coords: # Check if there are any particles to plot
            scatter_plot.set_offsets(np.c_[x_coords, y_coords])
        else: # No particles, set offsets to empty
            scatter_plot.set_offsets(np.empty((0, 2)))
            
        ax.set_title(f"FHP Simulation (Circular Particles) - Step {frame_num}")
        return [scatter_plot] # Return list of modified artists

    ani = animation.FuncAnimation(fig, update_frame, frames=len(lattice_states),
                                  interval=interval_ms, blit=True, repeat=False)
    try:
        print(f"Attempting to save animation as {filename}...")
        actual_fps = 1000.0 / interval_ms if interval_ms > 0 else 10.0 
        
        if filename.endswith(".mp4"):
            writer = animation.FFMpegWriter(fps=actual_fps, metadata=dict(artist='FHP-II Simulation'), bitrate=1800)
            ani.save(filename, writer=writer)
            print(f"Animation saved as {filename}. (Requires ffmpeg)")
        elif filename.endswith(".gif"):
            writer = animation.PillowWriter(fps=actual_fps)
            ani.save(filename, writer=writer)
            print(f"Animation saved as {filename}. (Requires Pillow)")
        else:
            print(f"Unsupported file extension: {filename}.")
            return
    except Exception as e:
        print(f"Error saving animation: {e}")
    finally:
        plt.close(fig)

# --- Main Execution ---
if __name__ == '__main__':
    if RANDOM_SEED is not None:
        print(f"Using random seed: {RANDOM_SEED}")
        random.seed(RANDOM_SEED)

    print(f"--- FHP-II Lattice Gas Simulation (Circular Particles) ---")
    print(f"Grid Dimensions: {GRID_ROWS}x{GRID_COLUMNS}")
    print(f"Particle Density Target: {NUM_PARTICLES_PERCENTAGE*100:.2f}% of total grid")
    # print(f"Initial Dot Radius Fraction: {INITIAL_DOT_RADIUS_FRACTION}") # No longer using dot
    print(f"Simulation Steps: {SIMULATION_STEPS}")
    print(f"Boundary Type: {BOUNDARY_TYPE}")
    print(f"Animation Output: {ANIMATION_FILENAME} (Interval: {ANIMATION_INTERVAL_MS}ms)")

    current_lattice_state = initialize_lattice(GRID_ROWS, GRID_COLUMNS, BOUNDARY_TYPE)
    
    # Populate with particles across the grid
    fill_lattice_randomly(
        current_lattice_state, 
        NUM_PARTICLES_PERCENTAGE, 
        GRID_ROWS, 
        GRID_COLUMNS, 
        BOUNDARY_TYPE
        # Removed dot_radius_fraction parameter
    )

    # Store full lattice states for animation using scatter plot
    lattice_history_for_animation: List[np.ndarray] = [current_lattice_state.copy()] 

    print("\nStarting simulation loop...")
    for step in range(SIMULATION_STEPS):
        current_lattice_state = simulation_step(current_lattice_state, GRID_ROWS, GRID_COLUMNS, BOUNDARY_TYPE)
        lattice_history_for_animation.append(current_lattice_state.copy()) # Store a copy of the full state
        
        if SIMULATION_STEPS == 0: break
        print_interval = max(1, SIMULATION_STEPS // 20)
        if (step + 1) % print_interval == 0 or (step + 1) == SIMULATION_STEPS:
            print(f"  Completed step {step + 1}/{SIMULATION_STEPS}")
    
    print("\nSimulation finished.")
    
    if lattice_history_for_animation:
        build_and_save_animation(
            lattice_history_for_animation, # Pass list of Node arrays
            GRID_ROWS,
            GRID_COLUMNS,
            filename=ANIMATION_FILENAME, 
            interval_ms=ANIMATION_INTERVAL_MS
        )
    else:
        print("No frames were generated for the animation.")

    print("\n--- Simulation Complete ---")


Using random seed: 42
--- FHP-II Lattice Gas Simulation (Circular Particles) ---
Grid Dimensions: 50x50
Particle Density Target: 5.00% of total grid
Simulation Steps: 100
Boundary Type: wall
Animation Output: fhp_simulation.gif (Interval: 300ms)
Initialized 691 particles across the grid (out of 13824 available slots, 5.00% target density).

Starting simulation loop...
  Completed step 5/100
  Completed step 10/100
  Completed step 15/100
  Completed step 20/100
  Completed step 25/100
  Completed step 30/100
  Completed step 35/100
  Completed step 40/100
  Completed step 45/100
  Completed step 50/100
  Completed step 55/100
  Completed step 60/100
  Completed step 65/100
  Completed step 70/100
  Completed step 75/100
  Completed step 80/100
  Completed step 85/100
  Completed step 90/100
  Completed step 95/100
  Completed step 100/100

Simulation finished.
Attempting to save animation as fhp_simulation.gif...
Animation saved as fhp_simulation.gif. (Requires Pillow)

--- Simulation 

In [20]:
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from typing import List, Tuple, Optional, Any
import math # Added back for sqrt

# --- Configuration Parameters ---
GRID_ROWS: int = 50
GRID_COLUMNS: int = 50
# Percentage of total (cell, direction) slots WITHIN THE INITIAL DOT to fill
NUM_PARTICLES_PERCENTAGE: float = 0.20 # Increased for a denser dot
# Fraction of min(GRID_ROWS, GRID_COLUMNS) for the initial dot radius
INITIAL_DOT_RADIUS_FRACTION: float = 0.25 

SIMULATION_STEPS: int = 100
ANIMATION_INTERVAL_MS: int = 150  # Slower movement
BOUNDARY_TYPE: str = 'wall'  # Options: 'wall' or 'periodic'

# Default to GIF to avoid common ffmpeg issues for users
ANIMATION_FILENAME: str = "fhp_simulation_v3_dot_circular_particles_red.gif" # Updated filename
# To save as MP4 (requires ffmpeg installed and in PATH):
# ANIMATION_FILENAME: str = "fhp_simulation_v3_dot_circular_particles_red.mp4"

# Set a seed for reproducible simulations. Set to None for random behavior.
RANDOM_SEED: Optional[int] = 42

# --- Node Class ---
class Node:
    """Represents a single cell in the lattice."""
    def __init__(self, is_wall: bool = False):
        self.ni_s: List[int] = [0] * 6
        self.is_wall: bool = is_wall
        # 'occupied' is primarily for visualization: 1 if wall or any particle present.
        self.occupied: int = 1 if is_wall else 0

    def __repr__(self) -> str:
        return f"Node(wall={self.is_wall}, ni_s={self.ni_s}, occupied={self.occupied})"

# --- Lattice Initialization ---
def initialize_lattice(rows: int, cols: int, boundary_type: str) -> np.ndarray:
    """Initializes the simulation lattice with Node objects."""
    lattice = np.array([[Node(is_wall=False) for _ in range(cols)] for _ in range(rows)], dtype=object)
    
    if boundary_type == 'wall':
        for r_idx in range(rows):
            for c_idx in range(cols):
                if r_idx == 0 or r_idx == rows - 1 or c_idx == 0 or c_idx == cols - 1:
                    lattice[r_idx, c_idx] = Node(is_wall=True)
    return lattice

# --- Neighbor Finding ---
def get_hexagonal_neighbors(coords: Tuple[int, int], rows: int, cols: int) -> List[Tuple[int, int]]:
    """Gets the 6 neighbors of a cell in a hexagonal layout on a square grid."""
    r, c = coords
    return [
        (r, (c + 1) % cols),
        ((r - 1 + rows) % rows, (c + 1) % cols),
        ((r - 1 + rows) % rows, (c - 1 + cols) % cols),
        (r, (c - 1 + cols) % cols),
        ((r + 1) % rows, (c - 1 + cols) % cols),
        ((r + 1) % rows, (c + 1) % cols)
    ]

# --- Particle Initialization ---
def fill_lattice_randomly(
    lattice: np.ndarray, 
    num_particles_percentage: float, 
    rows: int, 
    cols: int, 
    boundary_type: str,
    dot_radius_fraction: float # Added parameter for dot initialization
    ):
    """
    Fills the lattice with particles forming an initial round dot at the center.
    Particles occupy unique (cell_row, cell_col, direction_index) slots within the dot.
    The number of particles is determined by num_particles_percentage of available slots *within the dot*.
    """
    possible_slots: List[Tuple[int, int, int]] = []
    
    dot_center_r = rows // 2
    dot_center_c = cols // 2
    # Calculate radius based on the smaller dimension of the grid
    dot_radius = min(rows, cols) * dot_radius_fraction 
    
    print(f"Initializing particles in a dot at ({dot_center_r}, {dot_center_c}) with radius {dot_radius:.2f}.")

    for r_idx in range(rows):
        for c_idx in range(cols):
            # Skip wall cells if boundary_type is 'wall'
            if boundary_type == 'wall' and lattice[r_idx, c_idx].is_wall:
                continue
            
            # Check if the cell is within the dot radius
            distance_from_center = math.sqrt((r_idx - dot_center_r)**2 + (c_idx - dot_center_c)**2)
            
            if distance_from_center <= dot_radius:
                # For each cell within the dot, all 6 directions are potential slots
                for direction_idx in range(6):
                    possible_slots.append((r_idx, c_idx, direction_idx))
    
    if not possible_slots:
        print("Warning: No available slots within the defined dot to place particles (dot might be too small or outside non-wall area).")
        return

    num_particles_to_place = int(num_particles_percentage * len(possible_slots))
    # Ensure at least one particle if percentage > 0 and slots exist within the dot
    if num_particles_to_place == 0 and num_particles_percentage > 0 and possible_slots:
        num_particles_to_place = 1
        
    # Ensure we don't try to place more particles than available slots within the dot
    actual_num_particles = min(num_particles_to_place, len(possible_slots))

    if actual_num_particles > 0:
        chosen_slots = random.sample(possible_slots, actual_num_particles)
        for r_idx, c_idx, direction_idx in chosen_slots:
            lattice[r_idx, c_idx].ni_s[direction_idx] = 1
            lattice[r_idx, c_idx].occupied = 1 # Mark cell as occupied for visualization
    
    print(f"Initialized {actual_num_particles} particles within the dot (out of {len(possible_slots)} available slots in dot, {num_particles_percentage*100:.2f}% target density in dot).")


# --- FHP-II Collision Rules ---
def _d_predicate(ni_s: List[int], k: int) -> int:
    if ni_s[k % 6] == 1 and \
       ni_s[(k + 3) % 6] == 1 and \
       ni_s[(k + 1) % 6] == 0 and \
       ni_s[(k + 2) % 6] == 0 and \
       ni_s[(k + 4) % 6] == 0 and \
       ni_s[(k + 5) % 6] == 0:
        return 1
    return 0

def _t_predicate(ni_s: List[int], k: int) -> int:
    if ni_s[k % 6] == 1 and \
       ni_s[(k + 2) % 6] == 1 and \
       ni_s[(k + 4) % 6] == 1 and \
       ni_s[(k + 1) % 6] == 0 and \
       ni_s[(k + 3) % 6] == 0 and \
       ni_s[(k + 5) % 6] == 0:
        return 1
    return 0

def calculate_collision_factor_omega(ni_s: List[int], i: int) -> int:
    q = random.choice([0, 1])
    loss_from_d_collision_on_i_axis = _d_predicate(ni_s, i)
    gain_from_d_collision_on_i_minus_1_axis = q * _d_predicate(ni_s, (i - 1 + 6) % 6)
    gain_from_d_collision_on_i_plus_1_axis = (1 - q) * _d_predicate(ni_s, (i + 1 + 6) % 6)
    loss_from_t_collision_involving_i = _t_predicate(ni_s, i)
    gain_from_t_collision_involving_i_plus_3 = _t_predicate(ni_s, (i + 3 + 6) % 6)
    omega = (
        -loss_from_d_collision_on_i_axis +
        gain_from_d_collision_on_i_minus_1_axis +
        gain_from_d_collision_on_i_plus_1_axis -
        loss_from_t_collision_involving_i +
        gain_from_t_collision_involving_i_plus_3
    )
    return int(omega)

# --- Simulation Step ---
def simulation_step(current_lattice: np.ndarray, rows: int, cols: int, boundary_type: str) -> np.ndarray:
    next_lattice = initialize_lattice(rows, cols, boundary_type)

    for r_idx in range(rows):
        for c_idx in range(cols):
            current_node = current_lattice[r_idx, c_idx]

            if boundary_type == 'wall' and current_node.is_wall:
                continue 

            current_ni_s = current_node.ni_s
            ni_s_post_collision = [0] * 6

            for k_direction in range(6):
                omega_k = calculate_collision_factor_omega(current_ni_s, k_direction)
                particles_in_k_after_collision = current_ni_s[k_direction] + omega_k
                if particles_in_k_after_collision < 0: particles_in_k_after_collision = 0
                if particles_in_k_after_collision > 1: particles_in_k_after_collision = 1
                ni_s_post_collision[k_direction] = particles_in_k_after_collision
            
            neighbor_coords_list = get_hexagonal_neighbors((r_idx, c_idx), rows, cols)
            for k_direction in range(6):
                if ni_s_post_collision[k_direction] == 1: 
                    dest_r, dest_c = neighbor_coords_list[k_direction]
                    
                    if boundary_type == 'wall' and next_lattice[dest_r, dest_c].is_wall:
                        next_lattice[r_idx, c_idx].ni_s[(k_direction + 3) % 6] = 1
                    else:
                        next_lattice[dest_r, dest_c].ni_s[k_direction] = 1

    for r_idx in range(rows):
        for c_idx in range(cols):
            node_to_update = next_lattice[r_idx, c_idx]
            if any(particle_present == 1 for particle_present in node_to_update.ni_s):
                node_to_update.occupied = 1
            elif node_to_update.is_wall:
                node_to_update.occupied = 1 
            else:
                node_to_update.occupied = 0
            
    return next_lattice

# --- Visualization and Animation ---
def get_particle_coordinates(lattice: np.ndarray, rows: int, cols: int) -> Tuple[List[int], List[int]]:
    """Extracts x (column) and y (row) coordinates of cells containing particles."""
    x_coords: List[int] = []
    y_coords: List[int] = []
    for r_idx in range(rows):
        for c_idx in range(cols):
            if not lattice[r_idx, c_idx].is_wall and \
               any(p == 1 for p in lattice[r_idx, c_idx].ni_s): 
                x_coords.append(c_idx) 
                y_coords.append(r_idx) 
    return x_coords, y_coords

def display_lattice_state(lattice: np.ndarray, rows: int, cols: int, step_num: int = -1):
    """Displays a single state of the lattice using Matplotlib scatter plot for circular particles."""
    x_coords, y_coords = get_particle_coordinates(lattice, rows, cols)
    
    plt.figure(figsize=(7, 6))
    ax = plt.gca() 
    
    wall_grid = np.zeros((rows, cols), dtype=np.int8)
    for r_idx in range(rows):
        for c_idx in range(cols):
            if lattice[r_idx, c_idx].is_wall:
                 wall_grid[r_idx,c_idx] = 1
    ax.imshow(wall_grid, cmap='Greys', vmin=0, vmax=5, alpha=0.3, origin='upper', extent=[-0.5, cols-0.5, rows-0.5, -0.5])

    marker_area = (0.8 * min(700/cols, 600/rows))**2 
    ax.scatter(x_coords, y_coords, s=marker_area, c='red', marker='o', alpha=0.8)
    
    ax.set_xlim(-0.5, cols - 0.5)
    ax.set_ylim(rows - 0.5, -0.5) 
    ax.set_aspect('equal', adjustable='box') 

    title = "FHP Lattice State (Circular Particles)"
    if step_num >= 0:
        title += f" - Step {step_num}"
    ax.set_title(title)
    ax.set_xlabel("Column Index")
    ax.set_ylabel("Row Index")
    plt.tight_layout()
    plt.show()

def build_and_save_animation(
    lattice_states: List[np.ndarray], 
    rows: int,
    cols: int,
    filename: str = "fhp_simulation.gif", 
    interval_ms: int = 150,
    particle_color: str = 'red'
    ):
    """Builds and saves an animation from a list of lattice states using scatter plot."""
    if not lattice_states:
        print("Error: No states provided for animation.")
        return

    fig, ax = plt.subplots(figsize=(7, 6))
    
    marker_area = (0.8 * min(700/cols, 600/rows))**2 
    scatter_plot = ax.scatter([], [], s=marker_area, c=particle_color, marker='o', alpha=0.8)

    wall_grid_img = None
    if BOUNDARY_TYPE == 'wall': 
        wall_grid = np.zeros((rows, cols), dtype=np.int8)
        for r_idx in range(rows):
            for c_idx in range(cols):
                if lattice_states[0][r_idx, c_idx].is_wall: 
                    wall_grid[r_idx,c_idx] = 1
        wall_grid_img = ax.imshow(wall_grid, cmap='Greys', vmin=0, vmax=5, alpha=0.3, origin='upper', extent=[-0.5, cols-0.5, rows-0.5, -0.5])


    ax.set_xlim(-0.5, cols - 0.5)
    ax.set_ylim(rows - 0.5, -0.5) 
    ax.set_aspect('equal', adjustable='box')
    ax.set_title(f"FHP Simulation - Step 0")
    ax.set_xlabel("Column Index")
    ax.set_ylabel("Row Index")
    plt.tight_layout()

    def update_frame(frame_num: int):
        current_lattice = lattice_states[frame_num]
        x_coords, y_coords = get_particle_coordinates(current_lattice, rows, cols)
        
        if x_coords: 
            scatter_plot.set_offsets(np.c_[x_coords, y_coords])
        else: 
            scatter_plot.set_offsets(np.empty((0, 2)))
            
        ax.set_title(f"FHP Simulation (Circular Particles) - Step {frame_num}")
        
        return [scatter_plot] 

    ani = animation.FuncAnimation(fig, update_frame, frames=len(lattice_states),
                                  interval=interval_ms, blit=True, repeat=False)
    try:
        print(f"Attempting to save animation as {filename}...")
        actual_fps = 1000.0 / interval_ms if interval_ms > 0 else 10.0 
        
        if filename.endswith(".mp4"):
            writer = animation.FFMpegWriter(fps=actual_fps, metadata=dict(artist='FHP-II Simulation'), bitrate=1800)
            ani.save(filename, writer=writer)
            print(f"Animation saved as {filename}. (Requires ffmpeg)")
        elif filename.endswith(".gif"):
            writer = animation.PillowWriter(fps=actual_fps)
            ani.save(filename, writer=writer)
            print(f"Animation saved as {filename}. (Requires Pillow)")
        else:
            print(f"Unsupported file extension: {filename}.")
            return
    except Exception as e:
        print(f"Error saving animation: {e}")
    finally:
        plt.close(fig)

# --- Main Execution ---
if __name__ == '__main__':
    if RANDOM_SEED is not None:
        print(f"Using random seed: {RANDOM_SEED}")
        random.seed(RANDOM_SEED)

    print(f"--- FHP-II Lattice Gas Simulation (Circular Particles - Dot Initial) ---") 
    print(f"Grid Dimensions: {GRID_ROWS}x{GRID_COLUMNS}")
    print(f"Particle Density Target (within dot): {NUM_PARTICLES_PERCENTAGE*100:.2f}%")
    print(f"Initial Dot Radius Fraction: {INITIAL_DOT_RADIUS_FRACTION}")
    print(f"Simulation Steps: {SIMULATION_STEPS}")
    print(f"Boundary Type: {BOUNDARY_TYPE}")
    print(f"Animation Output: {ANIMATION_FILENAME} (Interval: {ANIMATION_INTERVAL_MS}ms)")

    current_lattice_state = initialize_lattice(GRID_ROWS, GRID_COLUMNS, BOUNDARY_TYPE)
    
    # Populate with particles in a dot shape
    fill_lattice_randomly(
        current_lattice_state, 
        NUM_PARTICLES_PERCENTAGE, 
        GRID_ROWS, 
        GRID_COLUMNS, 
        BOUNDARY_TYPE,
        INITIAL_DOT_RADIUS_FRACTION # Pass the new parameter
    )

    lattice_history_for_animation: List[np.ndarray] = [current_lattice_state.copy()] 

    print("\nStarting simulation loop...")
    for step in range(SIMULATION_STEPS):
        current_lattice_state = simulation_step(current_lattice_state, GRID_ROWS, GRID_COLUMNS, BOUNDARY_TYPE)
        lattice_history_for_animation.append(current_lattice_state.copy()) 
        
        if SIMULATION_STEPS == 0: break
        print_interval = max(1, SIMULATION_STEPS // 20)
        if (step + 1) % print_interval == 0 or (step + 1) == SIMULATION_STEPS:
            print(f"  Completed step {step + 1}/{SIMULATION_STEPS}")
    
    print("\nSimulation finished.")
    
    if lattice_history_for_animation:
        build_and_save_animation(
            lattice_history_for_animation, 
            GRID_ROWS,
            GRID_COLUMNS,
            filename=ANIMATION_FILENAME, 
            interval_ms=ANIMATION_INTERVAL_MS
        )
    else:
        print("No frames were generated for the animation.")

    print("\n--- Simulation Complete ---")


Using random seed: 42
--- FHP-II Lattice Gas Simulation (Circular Particles - Dot Initial) ---
Grid Dimensions: 50x50
Particle Density Target (within dot): 20.00%
Initial Dot Radius Fraction: 0.25
Simulation Steps: 100
Boundary Type: wall
Animation Output: fhp_simulation_v3_dot_circular_particles_red.gif (Interval: 150ms)
Initializing particles in a dot at (25, 25) with radius 12.50.
Initialized 586 particles within the dot (out of 2934 available slots in dot, 20.00% target density in dot).

Starting simulation loop...
  Completed step 5/100
  Completed step 10/100
  Completed step 15/100
  Completed step 20/100
  Completed step 25/100
  Completed step 30/100
  Completed step 35/100
  Completed step 40/100
  Completed step 45/100
  Completed step 50/100
  Completed step 55/100
  Completed step 60/100
  Completed step 65/100
  Completed step 70/100
  Completed step 75/100
  Completed step 80/100
  Completed step 85/100
  Completed step 90/100
  Completed step 95/100
  Completed step 100